 Week07-TextMining 

#### Team members : 
 Bapuji Satyala (U61600601)

In this Jupyter notebook, we have used the dataset from UCI repository " User review setiment Analysis " where our data consists of 3 files from 3 sources imdb,yelp and amazon. we have combined the 3 files into a single dataframe and performed text analysis on the same. Initially we haved applied text lemmetization to reduce the words to the base form, post that data is split to undergo TFID vectorization. TFIDF vectorization includes preprocessing of data,tokenisation and filtering stop words. the data undergoes Single value decompositon to lessen the chance of overfitting and speed up computation and fit into the respective model thus acheiving the performance metrics using Hyoerparameter tuning to acheive the best model analysis.

#### Importing the dataset from reliable source like UCI Repository.

#### Importing all the necessary Libraries

In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

random_seed = 42  # set random seed to ensure the results are repeatable.

 Our dataset contains user reviews from three different sources, lets combine them first to perform an analysis on the combined dataset.

In [18]:

file_names = ['yelp_labelled.txt','imdb_labelled.txt','amazon_cells_labelled.txt']
df_list=[]

for file_name in file_names:
    df_list.append(pd.read_csv(file_name, names=['sentence', 'label'], sep='\t'))

df_list

[                                              sentence  label
 0                             Wow... Loved this place.      1
 1                                   Crust is not good.      0
 2            Not tasty and the texture was just nasty.      0
 3    Stopped by during the late May bank holiday of...      1
 4    The selection on the menu was great and so wer...      1
 ..                                                 ...    ...
 995  I think food should have flavor and texture an...      0
 996                           Appetite instantly gone.      0
 997  Overall I was not impressed and would not go b...      0
 998  The whole experience was underwhelming, and I ...      0
 999  Then, as if I hadn't wasted enough of my life ...      0
 
 [1000 rows x 2 columns],
                                               sentence  label
 0    A very, very, very slow-moving, aimless movie ...      0
 1    Not sure who was more lost - the flat characte...      0
 2    Attempting artiness w

In above code reads labeled phrases from three text files (yelp_labelled.txt, imdb_labelled.txt, and amazon_cells_labelled.txt) into pandas DataFrames. Sentence and Label are the two columns that make up each DataFrame. The content of the sentence is located in the'sentence' column, while the binary label designating whether the sentence is positive (1) or negative (0) is located in the 'label' column. The DataFrames are then kept by the code in a list named df_list, where each entry points to a different input file. This method works well for loading several datasets into memory for preprocessing or additional analysis. Putting data in a structured format, such as pandas DataFrames, makes it easier to manipulate and analyze; this is a typical technique in data science and machine learning operations.

In [19]:
df=pd.concat(df_list)
df

,sentence,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [20]:
df[['sentence']].isna().sum()

sentence    0
dtype: int64

Checking the null values in Sentence.

## Perform text lemmatization to reduce words to their base or root form.

In [21]:
transformed_corpus = []
wnl = WordNetLemmatizer()
for document in df['sentence']:
    transformed_document = ""
    for word, tag in pos_tag(word_tokenize(document)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)
        transformed_document+= lemma + " "
    transformed_corpus += [transformed_document]

df['transformed_sentence']=transformed_corpus

df

,sentence,label,transformed_sentence
0,Wow... Loved this place.,1,Wow ... Loved this place .
1,Crust is not good.,0,Crust be not good .
2,Not tasty and the texture was just nasty.,0,Not tasty and the texture be just nasty .
3,Stopped by during the late May bank holiday of...,1,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...,1,The selection on the menu be great and so be t...
...,...,...,...
995,The screen does get smudged easily because it ...,0,The screen do get smudge easily because it tou...
996,What a piece of junk.. I lose more calls on th...,0,What a piece of junk .. I lose more call on th...
997,Item Does Not Match Picture.,0,Item Does Not Match Picture .
998,The only thing that disappoint me is the infra...,0,The only thing that disappoint me be the infra...


Text lemmatization, or breaking down words into their base or root form, is carried out by this code on the sentences included in the DataFrame df(sentence). NLTK library's WordNet Lemmatizer is used for the lemmatization process. The lemmatized phrases are initially stored in a blank list called transformed_corpus. By joining the lemmatized words together, it iteratively creates the changed phrase for each sentence in the DataFrame. To the list of converted corpuses, the transformed sentence is appended.
At the end, it creates a new column in the DataFrame named transformed_sentence that has the sentences' lemmatized forms.
Words are reduced to their most basic forms through the lemmatization process, which helps standardize the text data and enhances the efficiency of natural language processing activities like sentiment analysis and text classification.

#### Split the data

In [22]:
X = df['transformed_sentence']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

This code uses the train_test_split function from the scikit-learn library to divide the modified sentences (X) and their matching labels (y) into training and testing sets. We can train the model on a subset of the data and assess its performance on a different subset by dividing the dataset into training and testing sets. This helps prevent overfitting, which occurs when a model performs well on training data but badly on fresh, unseen data, and evaluates how well the model generalizes to new data. Thirty percent of the data will be utilized for testing and seventy percent will be used for training, according to the test_size=0.3 option. The size of the training and testing sets can be impacted by changing this parameter.

## TF-IDF vectorization

In [23]:
#TfidfVectorizer includes pre-processing, tokenization, filtering stop words
tfidf_vect = TfidfVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+")

X_train = tfidf_vect.fit_transform(X_train)
X_test = tfidf_vect.transform(X_test)

pd.DataFrame(X_train.toarray(), columns=tfidf_vect.get_feature_names_out())

,aailiyah,ability,able,abound,abroad,absolute,absolutel,absolutely,abstruse,abysmal,...,younger,youthful,youtube,yucky,yukon,yummy,z,zero,zombie,zombiez
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Above code segment uses scikit-learn's TF-IDF vectorizer (TfidfVectorizer) to eliminate stop words from the altered phrases. In natural language processing, TF-IDF vectorization is a widely used technique that turns text data into numerical vectors while accounting for the significance of each word in the corpus. In order to effectively train and evaluate machine learning models for tasks like sentiment analysis, classification, or any other predictive modeling activity, the text data must first undergo preprocessing and feature engineering.

## Apply Singular Value Decomposition (SVD)

In [24]:
svd = TruncatedSVD(n_components=300, n_iter=10) #n_components is the number of topics, which should be less than the number of features

X_train= svd.fit_transform(X_train)
X_test = svd.transform(X_test)

pd.DataFrame(X_train, columns=[f"svd{num:04}" for num in range(0,X_train.shape[1])])

,svd0000,svd0001,svd0002,svd0003,svd0004,svd0005,svd0006,svd0007,svd0008,svd0009,...,svd0290,svd0291,svd0292,svd0293,svd0294,svd0295,svd0296,svd0297,svd0298,svd0299
0,0.036036,0.004047,0.010477,-0.000212,0.023794,-0.024190,-0.005102,0.125046,-0.025465,-0.011163,...,0.021773,0.027602,-0.017960,-0.016527,-0.057679,0.040319,-0.016026,0.013869,0.004335,0.018942
1,0.012802,0.028359,0.012502,0.058239,0.027380,-0.046723,0.072406,0.030775,-0.006315,-0.009869,...,0.026913,-0.050628,0.006768,0.035223,-0.020398,-0.016046,-0.040581,0.024010,0.014855,0.015992
2,0.045713,0.091588,-0.020400,-0.052102,0.022160,-0.102036,0.114591,-0.016682,-0.070260,-0.048706,...,0.002968,0.026344,0.018515,-0.054598,-0.009361,0.059165,-0.019670,-0.007820,0.015184,-0.033229
3,0.036720,0.027728,0.027896,-0.018806,0.082960,-0.026933,-0.037530,0.011056,-0.012303,0.021355,...,-0.012977,0.023511,-0.006769,-0.032233,0.012983,0.028078,-0.002234,0.029054,0.013473,0.076427
4,0.055962,0.137946,0.112265,0.418814,0.041115,-0.015961,0.113732,-0.042769,-0.079189,-0.137569,...,0.009773,-0.010169,-0.022266,0.019233,0.008603,0.012213,0.004572,0.000153,-0.008426,0.020464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918,0.041307,0.103656,-0.096002,-0.073437,0.012566,-0.171944,0.227098,-0.008403,-0.000868,0.054963,...,0.027826,-0.036408,-0.000802,0.023307,-0.011283,0.014631,-0.001109,0.023998,0.004203,-0.010399
1919,0.056659,0.024515,0.028658,0.029943,0.035624,-0.034223,-0.005631,0.198052,-0.048357,-0.040169,...,-0.014674,-0.003184,0.015108,0.004208,0.003222,0.009626,0.031377,0.008078,0.001283,0.001157
1920,0.074759,-0.007846,0.020618,0.007871,0.033606,-0.039924,-0.031925,0.289709,-0.056566,0.013666,...,-0.033511,0.023786,0.018251,-0.040950,0.008795,0.011306,0.008002,-0.006215,-0.027792,-0.011165
1921,0.044818,0.053056,0.055661,0.064574,-0.012860,0.009161,0.005199,0.017559,0.058988,0.129803,...,0.007720,-0.005633,-0.017176,0.013230,-0.038849,0.067878,-0.004271,-0.005677,0.007262,0.014433


Singular Value Decomposition (SVD) is used in this code segment to reduce dimensionality and enhance model performance. SVD helps lessen the chance of overfitting and speed up computation by decreasing the dimensionality of the data while keeping the most important information. 


n_components is set to 300 to  to strike a balance between computational efficiency and the amount of variance from the original data that is retained in the reduced dataset

# Data Modelling 

## Logistic Regression

In [25]:
logreg_model = LogisticRegression()
logreg_model.fit(X_train,y_train)

y_pred_log = logreg_model.predict(X_test)

print("************************************")
print('Performace of Logistic Regression')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_log)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_log)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_log)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_log)}")
print("************************************")


param_grid_random = {
    'C': np.logspace(-3, 3, 50), 
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'max_iter': [100, 500],  
}

best_random_search_model_log = RandomizedSearchCV(
    estimator=LogisticRegression(random_state=random_seed), 
    scoring='accuracy', 
    param_distributions=param_grid_random, 
    n_iter=100,  
    cv=10, 
    verbose=0, 
    n_jobs=-1
)

_ = best_random_search_model_log.fit(X_train, y_train)


y_pred = best_random_search_model_log.predict(X_test)
print("************************************")
print('Performace of Logistic Regression using Random Search')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")
print("************************************")

************************************
Performace of Logistic Regression

Accuracy Score:   0.7527272727272727
Recall Score:     0.7377049180327869
Precision Score:  0.773955773955774
F1 Score:         0.7553956834532375
************************************
************************************
Performace of Logistic Regression using Random Search

Accuracy Score:   0.7442424242424243
Recall Score:     0.7072599531615925
Precision Score:  0.7783505154639175
F1 Score:         0.7411042944785277
************************************


Logistic Regression model is trained without the need for hyperparameter tuning, and its performance is assessed using measures like accuracy, recall, precision, and F1 score.

Next, it uses Randomized Search Cross-Validation to find the optimal set of Logistic Regression model hyperparameters. The model is retrained with the ideal hyperparameters following hyperparameter adjustment, and it is assessed once more using the same performance criteria.

The benefit of hyperparameter optimization in enhancing model performance is demonstrated by comparing the performance of Logistic Regression with and without hyperparameter tuning.

Overall, the code shows how to construct and optimize a Logistic Regression model for binary classification tasks in a methodical manner, emphasizing the significance of performance evaluation, hyperparameter adjustment, and preprocessing in machine learning processes.

- Upon hyperparameter tuning the logistic regression accuracy has decreased from 0.784 to 0.7818.
- The recall score remained the same for logistic regression has increased from 0.7528 to 0.7597 after hyperparameter tuning.
- The precision score of the logistic regression has decreased from 0.8245 to 0.8157 after hyper parameter tuning.
- The F1 score has improved slightly from 0.7870 to 0.7867 after hyperparameter tuning.



# SVM

In [ ]:
svm_model= SVC()
svm_model.fit(X_train,y_train)

y_pred_svm = svm_model.predict(X_test)

print("************************************")
print('Performace of SVM')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_svm)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_svm)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_svm)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_svm)}")
print("************************************")

param_grid_random = {
    'C': np.logspace(-3, 2, 10),    
    'kernel': ['linear', 'rbf', 'poly'], 
    'gamma': np.logspace(-3, 2, 10)     
}

best_random_search_model_svm = RandomizedSearchCV(
    estimator=SVC(random_state=random_seed), 
    scoring='accuracy', 
    param_distributions=param_grid_random, 
    n_iter=100, 
    cv=10, 
    verbose=0, 
    n_jobs=-1
)

_ = best_random_search_model_svm.fit(X_train, y_train)


y_pred = best_random_search_model_svm.predict(X_test)
print("************************************")
print('Performace of SVM using Random Search')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")
print("************************************")

************************************
Performace of SVM

Accuracy Score:   0.7551515151515151
Recall Score:     0.7564402810304449
Precision Score:  0.7672209026128266
F1 Score:         0.7617924528301887
************************************


Fits the SVM model to the training data (X_train, y_train) after initializing it with SVC(). uses the trained SVM model to predict the target values for the test data (X_test).
calculates and prints metrics like Accuracy Score, Recall Score, Precision Score, and F1 Score to assess the effectiveness of the basic SVM model.

Hyperparameter Tuning with Randomized Search is done crating a a parameter grid (param_grid_random) with various values for hyperparameters such as "C," "kernel," and "gamma." establishes a Randomized Search Cross-Validation (RandomizedSearchCV) to determine the SVM model's optimal hyperparameters.

- Upon hyperparameter tuning the SVM accuracy has increased from 0.764 to 0.775.
- The recall score of the SVM has increased from 0.7438 to 0.7807 after hyper parameter tuning.
- The precision score of the SVM has decreased from 0.7704 to 0.7675 after hyper parameter tuning.
- The F1 score has improved slightly from 0.7568 to 0.7741 after hyperparameter tuning.

# Decision Tree

In [ ]:
dtree_model = DecisionTreeClassifier()
dtree_model.fit(X_train,y_train)

y_pred_dtree = dtree_model.predict(X_test)

print("************************************")
print('Performace of Decision Tree ')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_dtree)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_dtree)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_dtree)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_dtree)}")
print("************************************")


criterion = ['gini', 'entropy']
max_depth = [int(x) for x in np.linspace(1, 1000, 20)] 
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(2, 100, 20)]  
min_samples_leaf = [int(x) for x in np.linspace(1, 50, 20)]  
max_leaf_nodes = [int(x) for x in np.linspace(2, 100, 20)]  
max_leaf_nodes.append(None)
min_impurity_decrease = [x for x in np.arange(0.0, 0.01, 0.0001).round(5)]  

param_grid_random = {
    'criterion': criterion,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf' : min_samples_leaf,
    'max_leaf_nodes' : max_leaf_nodes,
    'min_impurity_decrease' : min_impurity_decrease,
}

best_random_search_model_dt = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(random_state=random_seed), 
    scoring='accuracy', 
    param_distributions=param_grid_random, 
    n_iter=100,
    cv=10, 
    verbose=0, 
    n_jobs=-1
)

_ = best_random_search_model_dt.fit(X_train, y_train)




y_pred = best_random_search_model_dt.predict(X_test)
print("************************************")
print('Performace of Decision Tree using Random Search')
print()
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")
print("************************************")

A Decision Tree is a flowchart-like tree structure where an internal node represents a feature(or attribute), the branch represents a decision rule, and each leaf node represents the outcome. Using the training data, they derive basic decision-making rules to forecast the target variable.

Hyperparameters are those that are chosen before the model is trained. The process of hyperparameter tuning entails determining the best set of hyperparameters to increase model performance.
Techniques:
Randomized Search: Selects hyperparameters at random from predefined distributions and assesses how well they work.

Hyperparameters of Decision Trees:
Criteria: Indicates a split's quality (e.g., entropy or Gini impurity).
Max Depth: The tree's maximum depth.
The bare minimum of samples needed to separate an internal node is known as "Min Samples Split."
Min Leaf Samples: The bare minimum of samples necessary for a leaf node.
Maximum Leaf Nodes: The highest quantity of leaf nodes within the tree.
Min Impurity Decrease: The least amount of impurities needed for a split to occur

- Upon hyperparameter tuning the Decision tree accuracy has increased from 0.64 to 0.704.
- The recall score of the decision tree has decreased from 0.635 to 0.632 after hyper parameter tuning.
- The precision score of the decision tree has decreased from 0.6339 to 0.6098 after hyper parameter tuning.
- The F1 score has improved slightly from 0.6346 to 0.6537 after hyperparameter tuning the model.





## KNN

In [12]:
knn_model = KNeighborsClassifier()

knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

print("************************************")
print('Performace of KNN ')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_knn)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_knn)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_knn)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_knn)}")
print("************************************")

param_grid_knn = {
    'n_neighbors': range(1, 21), 
    'weights': ['uniform', 'distance'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
    'p': [1, 2]  
}

best_random_search_model_knn = RandomizedSearchCV(
    estimator=KNeighborsClassifier(), 
    param_distributions=param_grid_knn, 
    n_iter=100,  
    cv=10, 
    scoring='accuracy', 
    verbose=0, 
    n_jobs=-1
)

_ = best_random_search_model_knn.fit(X_train, y_train)

y_pred_knn_best = best_random_search_model_knn.predict(X_test)
# Print performance metrics
print("************************************")
print('Performance of K-Nearest Neighbors using Random Search')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_knn_best)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_knn_best)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_knn_best)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_knn_best)}")
print("************************************")


************************************
Performace of KNN 

Accuracy Score:   0.6921212121212121
Recall Score:     0.679814385150812
Precision Score:  0.7163814180929096
F1 Score:         0.6976190476190476
************************************
************************************
Performance of K-Nearest Neighbors using Random Search

Accuracy Score:   0.7054545454545454
Recall Score:     0.6937354988399071
Precision Score:  0.7292682926829268
F1 Score:         0.7110582639714625
************************************


The similarity concept forms the foundation of KNN. It is predicated on the idea that data points with comparable numerical values or classes tend to be similar.
KNN uses the 'k' closest data points (nearest neighbors) in the feature space to generate predictions for a new data point. For classification purposes, this means assigning the majority class, and for regression, the average value of these neighbors to the new data point.

The KNN model's ideal hyperparameters are found via randomized search.
The weight function (weights), the technique used to find the nearest neighbors (algorithm), the power parameter for the Minkowski distance ('p'), and the number of neighbors ('n_neighbors') are examples of hyperparameters.

The test dataset's instance classification accuracy is used to evaluate the KNN model's performance.

- Upon hyperparameter tuning the KNN accuracy has increased slightly from 0.6436 to 0.6545.
- The recall score of the KNN has increased from 0.5246 to 0.5714 after hyper parameter tuning.
- The precision score of the KNN has decreased slightly from 0.6783 to 0.6763 after hyper parameter tuning.
- The F1 score has improved  from 0.5916 to 0.6194 after hyperparameter tuning the model.



## AdaBoost

In [13]:
adaboost_model = AdaBoostClassifier()

adaboost_model.fit(X_train, y_train)

y_pred_adaboost = adaboost_model.predict(X_test)

print("************************************")
print('Performace of AdaBoost ')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_adaboost)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_adaboost)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_adaboost)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_adaboost)}")
print("************************************")

param_grid_adaboost = {
    'n_estimators': [50, 100, 200, 300],  
    'learning_rate': np.logspace(-3, 0, 50), 
    'algorithm': ['SAMME', 'SAMME.R']  
}


best_random_search_model_adaboost = RandomizedSearchCV(
    estimator=AdaBoostClassifier(), 
    param_distributions=param_grid_adaboost, 
    n_iter=50,  
    cv=5, 
    scoring='accuracy', 
    verbose=0, 
    n_jobs=-1
)

_ = best_random_search_model_adaboost.fit(X_train, y_train)


y_pred_adaboost_best = best_random_search_model_adaboost.predict(X_test)

print("************************************")
print('Performance of AdaBoost using Random Search')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_adaboost_best)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_adaboost_best)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_adaboost_best)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_adaboost_best)}")
print("************************************")


************************************
Performace of AdaBoost 

Accuracy Score:   0.6957575757575758
Recall Score:     0.6751740139211136
Precision Score:  0.7238805970149254
F1 Score:         0.6986794717887154
************************************


KeyboardInterrupt: 

AdaBoost is an ensemble learning technique that builds a powerful classifier by combining several weak learners, usually decision trees.
It operates by fitting a sequence of weak learners in a sequential manner on the training set, with each new learner concentrating on the cases that the earlier ones misclassified.

At the beginning, every instance in the training set has the same weight.
Using the weighted training data, a weak learner is trained and its performance assessed.
Higher weights are given to cases that the weak learner incorrectly classifies, while lower weights are given to examples that are successfully classified.
Each succeeding weak learner concentrates more on the previously incorrectly classified cases when the process is repeated a predetermined number of times (or until the required accuracy is attained).

The main hyperparameters of AdaBoost include the number of weak learners (n_estimators = 50, 100, 200, 300) and the learning rate (learning_rate : np.logspace(-3, 0, 50), ), which controls the contribution of each weak learner to the final prediction.

The performance of the Model is stated below -

- Upon hyperparameter tuning the Adaboost accuracy has increased from 0.6945 to 0.724.
- The recall score of the Adaboost has increased from 0.677 to 0.714 after hyper parameter tuning.
- The precision score of the Adaboost has increased from 0.6944 to 0.7231 after hyper parameter tuning.
- The F1 score has improved slightly from 0.6857 to 0.7187 after hyperparameter tuning the model.

## XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier()

xgb_model.fit(X_train, y_train)


y_pred_xgb = xgb_model.predict(X_test)


print("************************************")
print('Performance of AdaBoost ')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_xgb)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_xgb)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_xgb)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_xgb)}")
print("************************************")


param_grid_xgb = {
    'n_estimators': [50, 100, 200, 300], 
    'learning_rate': np.logspace(-3, 0, 50),  
    'max_depth': [3, 4, 5, 6],  
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], 
    'gamma': [0, 0.1, 0.2, 0.3, 0.4]  
}

best_random_search_model_xgb = RandomizedSearchCV(
    estimator=xgb.XGBClassifier(), 
    param_distributions=param_grid_xgb, 
    n_iter=50,  
    cv=5, 
    scoring='accuracy', 
    verbose=0, 
    n_jobs=-1
)

_ = best_random_search_model_xgb.fit(X_train, y_train)


y_pred_xgb_best = best_random_search_model_xgb.predict(X_test)

print("************************************")
print('Performance of XGBoost using Random Search')
print()
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred_xgb_best)}")
print(f"{'Recall Score:':18}{recall_score(y_test, y_pred_xgb_best)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred_xgb_best)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred_xgb_best)}")
print("************************************")


XGBoost is an ensemble learning technique that builds predictive models, primarily decision trees, using a gradient boosting framework.
With a focus on speed and performance, it is an extremely effective and streamlined version of gradient boosting.

Gradient Boosting: XGBoost trains a sequence of decision trees one after the other in an effort to fix the mistakes the earlier trees made.
The objective function of XGBoost is to minimize a given loss function by adding new trees that lower the residual errors. Typically, this loss function is the log loss for classification problems and the mean squared error for regression problems.
Regularization: To manage model complexity and avoid overfitting, XGBoost incorporates L1 (Lasso) and L2 (Ridge) regularization terms.

The number of trees (n_estimators: 50, 100, 200, 300), the learning rate (learning_rate:  np.logspace(-3, 0, 50)), which regulates each tree's contribution, and the maximum depth of each tree (max_depth: 3, 4, 5, 6) are important hyperparameters.
Subsample (the percentage of samples used to train each tree), colsample_bytree (the percentage of features used to train each tree), and gamma (the minimal loss reduction needed to perform a subsequent partition) are additional critical hyperparameters.

- Upon hyperparameter tuning the XGboost accuracy has increased from 0.7224 to 0.7406.
- The recall score of the XGboost has increased slightly from 0.7093 to 0.7345 after hyper parameter tuning.
- The precision score of the XGboost has increased slightly from 0.7524 to 0.7661 after hyper parameter tuning.
- The F1 score has decreased slightly from 0.7302 to 0.75 after hyperparameter tuning the model.

## Summary

The three files were merged into a single dataframe, and text analysis was done on it. After first using text lemmatization to condense the words to their most basic form, the data is divided and subjected to TFID vectorization. Tokenization, stop word filtering, and data preparation are all included in TFIDF vectorization. To reduce the likelihood of overfitting, expedite computing, and fit the data into the appropriate model, the data is subjected to single value decomposition. This allows for the achievement of performance metrics through the use of hybrid parameter tweaking, which yields the optimal model analysis.

#### Logistic Regression

- Upon hyperparameter tuning the logistic regression accuracy has decreased from 0.784 to 0.7818.
- The recall score remained the same for logistic regression has increased from 0.7528 to 0.7597 after hyperparameter tuning.
- The precision score of the logistic regression has decreased from 0.8245 to 0.8157 after hyper parameter tuning.
- The F1 score has improved slightly from 0.7870 to 0.7867 after hyperparameter tuning.

#### SVM

- Upon hyperparameter tuning the SVM accuracy has increased from 0.764 to 0.775.
- The recall score of the SVM has increased from 0.7438 to 0.7807 after hyper parameter tuning.
- The precision score of the SVM has decreased from 0.7704 to 0.7675 after hyper parameter tuning.
- The F1 score has improved slightly from 0.7568 to 0.7741 after hyperparameter tuning.

#### Decision Tree

- Upon hyperparameter tuning the Decision tree accuracy has increased from 0.64 to 0.704.
- The recall score of the decision tree has decreased from 0.635 to 0.632 after hyper parameter tuning.
- The precision score of the decision tree has decreased from 0.6339 to 0.6098 after hyper parameter tuning.
- The F1 score has improved slightly from 0.6346 to 0.6537 after hyperparameter tuning the model.


#### KNN

- Upon hyperparameter tuning the KNN accuracy has increased slightly from 0.6436 to 0.6545.
- The recall score of the KNN has increased from 0.5246 to 0.5714 after hyper parameter tuning.
- The precision score of the KNN has decreased slightly from 0.6783 to 0.6763 after hyper parameter tuning.
- The F1 score has improved  from 0.5916 to 0.6194 after hyperparameter tuning the model.

#### Adaboost

- Upon hyperparameter tuning the Adaboost accuracy has increased from 0.6945 to 0.724.
- The recall score of the Adaboost has increased from 0.677 to 0.714 after hyper parameter tuning.
- The precision score of the Adaboost has increased from 0.6944 to 0.7231 after hyper parameter tuning.
- The F1 score has improved slightly from 0.6857 to 0.7187 after hyperparameter tuning the model.

#### XGboost

- Upon hyperparameter tuning the XGboost accuracy has increased from 0.7224 to 0.7406.
- The recall score of the XGboost has increased slightly from 0.7093 to 0.7345 after hyper parameter tuning.
- The precision score of the XGboost has increased slightly from 0.7524 to 0.7661 after hyper parameter tuning.
- The F1 score has decreased slightly from 0.7302 to 0.75 after hyperparameter tuning the model.



Since we are practically using data from three different sources the accuracies of the models is maximized at 0.7818 by logistic regression with hyperparameter tuning thus indicating the complexity of model optimization when interpretability is considered.

XGBoost emerges as the best model overall, given its across-the-board improvements in accuracy, recall, precision, and F1 score.This suggests it is a robust model capable of handling various types of data and making accurate predictions with a good balance between identifying positive cases and minimizing false positives. 